##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Título

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/bigtable"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a> </td>
</table>

## Overview

En este cuaderno se representa el uso y las características básicos del módulo `tensorflow_io.bigtable`. Asegúrese de estar familiarizado con estos temas antes de continuar:

1. [Crear un proyecto de GCP.](https://console.cloud.google.com/cloud-resource-manager)
2. [Instalar Cloud SDK para Bigtable](https://cloud.google.com/bigtable/docs/installing-cloud-sdk)
3. [Descripción general de la herramienta cbt](https://cloud.google.com/bigtable/docs/cbt-overview)
4. [Usar el emulador](https://cloud.google.com/bigtable/docs/emulator)

Nota: Jupyter ejecuta líneas con <code>!</code> por delante como comandos de shell e interpola variables de Python con `$` por delante en estos comandos.

## Preparación

In [ ]:
!pip install tensorflow-io

**Nota**: Al ejecutar la siguiente celda, se le pedirá que inicie sesión en Google Cloud.

In [ ]:
!mkdir /tools/google-cloud-sdk/.install
!gcloud --quiet components install beta cbt bigtable
!gcloud init

Para poder ejecutar este ejemplo, se usa el emulador bigtable. Si tiene su instancia de bigtable configurada y llena de valores, omita estos pasos y vaya directamente a la sección de Guía rápida.

Inicie el emulador en segundo plano.

In [ ]:
import os
import subprocess
_emulator = subprocess.Popen(['/tools/google-cloud-sdk/bin/gcloud', 'beta', 'emulators', 'bigtable', 'start', '--host-port=127.0.0.1:8086'],
                                      stdout=subprocess.DEVNULL,
                                      stderr=subprocess.DEVNULL, bufsize=0)

Cree una tabla

In [ ]:
%env BIGTABLE_EMULATOR_HOST=127.0.0.1:8086
!cbt -project "test-project" -instance "test-instance" createtable t1 families=cf1 splits=row-a,row-h,row-p,row-z
!cbt -project "test-project" -instance "test-instance" ls


Rellene la tabla con valores

In [ ]:
!cbt -project "test-project" -instance "test-instance" set t1 row-a cf1:c1=A
!cbt -project "test-project" -instance "test-instance" set t1 row-b cf1:c1=B
!cbt -project "test-project" -instance "test-instance" set t1 row-c cf1:c1=C
!cbt -project "test-project" -instance "test-instance" set t1 row-d cf1:c1=D
!cbt -project "test-project" -instance "test-instance" set t1 row-e cf1:c1=E
!cbt -project "test-project" -instance "test-instance" set t1 row-f cf1:c1=F
!cbt -project "test-project" -instance "test-instance" set t1 row-g cf1:c1=G
!cbt -project "test-project" -instance "test-instance" set t1 row-h cf1:c1=H
!cbt -project "test-project" -instance "test-instance" set t1 row-i cf1:c1=I
!cbt -project "test-project" -instance "test-instance" set t1 row-j cf1:c1=J
!cbt -project "test-project" -instance "test-instance" set t1 row-k cf1:c1=K
!cbt -project "test-project" -instance "test-instance" set t1 row-l cf1:c1=L
!cbt -project "test-project" -instance "test-instance" set t1 row-m cf1:c1=M
!cbt -project "test-project" -instance "test-instance" set t1 row-n cf1:c1=N
!cbt -project "test-project" -instance "test-instance" set t1 row-o cf1:c1=O
!cbt -project "test-project" -instance "test-instance" set t1 row-p cf1:c1=P
!cbt -project "test-project" -instance "test-instance" set t1 row-q cf1:c1=Q
!cbt -project "test-project" -instance "test-instance" set t1 row-r cf1:c1=R
!cbt -project "test-project" -instance "test-instance" set t1 row-s cf1:c1=S
!cbt -project "test-project" -instance "test-instance" set t1 row-t cf1:c1=T
!cbt -project "test-project" -instance "test-instance" set t1 row-u cf1:c1=U
!cbt -project "test-project" -instance "test-instance" set t1 row-v cf1:c1=V
!cbt -project "test-project" -instance "test-instance" set t1 row-w cf1:c1=W
!cbt -project "test-project" -instance "test-instance" set t1 row-x cf1:c1=X
!cbt -project "test-project" -instance "test-instance" set t1 row-y cf1:c1=Y
!cbt -project "test-project" -instance "test-instance" set t1 row-z cf1:c1=Z

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_io as tfio
import random

random.seed(10)

## Guía rápida

Primero necesita crear un cliente y una tabla desde donde le gustaría leer los datos.

In [ ]:
# If using your bigtable instance replace the project_id, instance_id 
# and the name of the table with suitable values.

client = tfio.bigtable.BigtableClient(project_id="test-project", instance_id="test-instance")
train_table = client.get_table("t1")

¡Excelente! Ahora puede crear un conjunto de datos de TensorFlow que leerá los datos de nuestra tabla.

Para hacer eso, debe proporcionar el tipo de datos que desea leer, una lista de nombres de columnas en formato `column_family:column_name` y un conjunto de filas que desea leer.

Para crear un conjunto de filas, use los métodos de utilidad proporcionados en los módulos `tensorflow.bigtable.row_set` y `tensorflow.bigtable.row_range`. Aquí se crea un row_set que contiene todas las filas creadas.

Tenga en cuenta que esa tabla grande lee los valores en orden lexicográfico, no en el orden en que fueron agregados. Las filas recibieron claves de fila aleatorias para que se aleatorizen.

In [ ]:
row_set = tfio.bigtable.row_set.from_rows_or_ranges(tfio.bigtable.row_range.infinite())

train_dataset = train_table.read_rows(["cf1:c1"],row_set, output_type=tf.string)

for tensor in train_dataset:
  print(tensor)

¡Listo! ¡Felicitaciones!

## Lectura paralela

Nuestro conjunto de datos admite la lectura en paralelo desde Bigtable. Para hacer eso, use el método `parallel_read_rows` y especifique `num_parallel_calls` como argumento. Cuando se llama a este método, el trabajo se divide primero entre los trabajadores según SampleRowKeys.

**Nota**: Tenga en cuenta que al leer en paralelo, las filas no se leerán en ningún orden en particular.

In [ ]:
for tensor in train_table.parallel_read_rows(["cf1:c1"],row_set=row_set, num_parallel_calls=2):
  print(tensor)

## Lectura de row_keys específicas

Para leer los datos de Bigtable, puede especificar un conjunto de filas o un rango o una combinación de ellos.

El método `read_rows` espera que usted proporcione un RowSet. Puede construir un RowSet a partir de claves de fila específicas o RowRanges de la siguiente manera:

In [ ]:
row_range_below_300 = tfio.bigtable.row_range.right_open("row000", "row300")

my_row_set = tfio.bigtable.row_set.from_rows_or_ranges(row_range_below_300, "row585", "row832")
print(my_row_set)

tal row_set tendría un rango de filas `[row000, row300)` y filas row585 y row832.

También puede crear un row_set a partir de un rango infinito, un rango vacío o un prefijo. También puede cruzarlo con un row_range.

In [ ]:
my_truncated_row_set = tfio.bigtable.row_set.intersect(my_row_set,
                                         tfio.bigtable.row_range.right_open("row200", "row700"))
print(my_truncated_row_set)

## Especificar una versión de un valor

Bigtable le permite mantener muchos valores en una celda con diferentes marcas de tiempo. Puede especificar qué versión desea elegir con los filtros de versión. Sin embargo, solo puede recuperar un vector bidimensional con el conector tensorflow.bigtable, por lo que el filtro `latest` siempre se agrega al filtro de versión especificado por el usuario. Es decir, si más de un valor para una celda pasa por el filtro proporcionado, se usará el más nuevo.

Puede usar el filtro `latest` que pasa el valor más reciente o puede especificar un rango de tiempo. El rango de tiempo se puede proporcionar como objetos de fecha y hora de Python o como un número que represente segundos o microsegundos a partir de la época.

In [ ]:
from datetime import datetime

start = datetime(2020, 10, 10, 12, 0, 0)
end = datetime(2100, 10, 10, 13, 0, 0)
from_datetime = tfio.bigtable.filters.timestamp_range(start, end)
from_posix_timestamp = tfio.bigtable.filters.timestamp_range(int(start.timestamp()), int(end.timestamp()))
print("from_datetime:", from_datetime)

print("from_posix_timestamp:", from_posix_timestamp)